# JS 如何通过事件循环机制（EventLoop）实现异步执行

<img src='https://raw.githubusercontent.com/binghuis/assets/main/excalidraw/eventloop.avif' width='860px' />

https://www.jsv9000.app/

JS 是一种单线程的同步语言，这意味着在任何时候，JS 引擎中的调用栈只能执行一个任务。

调用栈是一个后进先出的栈结构，存在于 JS 引擎，用于管理 JS 执行上下文的执行顺序。

之前文章介绍过：

> 当 V8 执行全局、模块或函数代码时，在创建阶段会生成相应的执行上下文。执行上下文的状态组件「变量环境组件」和「词法环境组件」共同记录当前词法环境中的变量和函数。

当一个新的执行上下文被创建时，它会被压入调用栈，JS 引擎总是执行栈顶的上下文。当当前上下文的代码执行完毕后，它会从调用栈中弹出，控制权返回到上一个上下文。如果某个函数创建了闭包，尽管该函数的执行上下文在执行完毕后会从调用栈中弹出，但其词法环境仍会被闭包引用，因此不会被垃圾回收机制立即回收。

<img src='https://raw.githubusercontent.com/binghuis/assets/main/excalidraw/ec.avif' width='860px' />


调用栈一次只能处理一个任务，并且任务是同步执行的，但在日常开发中，为什么我们仍然可以使用异步操作？

处理异步操作在于将异步任务的处理从主线程中分离出来，通过事件循环机制和任务队列来管理和执行这些异步任务。这样，主线程可以继续执行其他同步任务，而异步操作在完成后，其回调函数会被放入任务队列中。当主线程空闲时，事件循环机制会将这些回调函数从任务队列中取出，放入调用栈中执行。

在深入了解事件循环机制之前，我们先了解一些和它有关的概念。


### 运行时（runtime）

JS 运行时指的是 JS 代码执行时的环境，它包括 JS 引擎（调用栈、内存堆、任务队列、事件循环机制）和宿主环境（宿主 API）。

#### 宿主 API

**宿主环境**通常指的是浏览器、Node.js 等宏观的 JS 执行环境，而运行时则是 JS 代码实际执行的基本运行环境。虽然 JS 引擎本质上是宿主环境的一部分，但由于其功能相对独立，所以在描述时往往将 JS 引擎与宿主环境并列列举，以便更好地理解其在 JS 执行中的角色。

> 当你浏览社区的时候你会发现，在社区讨论中，很多人默认将宿主环境和运行时环境视为同一个概念。

不同的宿主环境提供了各自独特的宿主 API：

- **浏览器宿主 API**：包括用于操作网页内容的 DOM、用于处理异步操作的 Promise 以及其他功能如定时器等。
- **Node.js 宿主 API**：包括用于处理文件系统的 `fs`、用于处理网络请求的 `http` 以及其他功能如定时器等。

它们 为 JS 提供了额外的能力，使开发者在不同环境能实现特定的功能。

#### 任务队列

任务队列是 JS 引擎用于存放待执行的任务的队列结构，分为任务队列和微任务队列：

- 任务：包含整体代码、setTimeout、setInterval、I/O 操作等。这些任务通常较大，会在事件循环的每一轮中被处理。
- 微任务：包含 Promise 的 then 和 catch 回调、MutationObserver 等。这些任务通常较小，会在当前任务完成后、下一个任务开始之前被处理。

#### 内存堆

JS 引擎通过调用栈和堆执行代码和管理内存。

- 调用栈：
- 内存堆：

在执行上下文里，基本类型的变量值存在环境记录里，引用类型和函数的值存在堆内存里，环境记录里存的是堆的内存地址。

相关概念我们已经了解了，下面是事件循环机制的详细过程：
